Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
from sklearn.metrics import roc_curve, auc
from google.colab import files
from tensorflow.keras.preprocessing import image

Code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/input/Testing/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk('/content/drive/MyDrive/input/Training/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/input/Testing/meningioma_tumor/image(10).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(12).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(13).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(120).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(16).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(27).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(1).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(25).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(14).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(102).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(28).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(32).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(15).jpg
/content/drive/MyDrive/input/Testing/meningioma_tumor/image(18).jpg
/content/drive/MyDrive/input/Testing/meningioma

In [ ]:
X_train = []
Y_train = []
image_size = 150
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/input/Training',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/input/Testing',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        Y_train.append(i)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
X_train,Y_train = shuffle(X_train,Y_train,random_state=101)
X_train.shape

(3264, 150, 150, 3)

Test Train Split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_train,Y_train,test_size=0.20,random_state=42)

In [ ]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train=y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test=y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

Convolutional Neural Network

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(150,150,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(4,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
history = model.fit(X_train,y_train,epochs=30,validation_split=0.1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 73, 73, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 73, 73, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 69, 69, 64)        36928     
                                                        

VGG

In [ ]:
# Import VGG16 from Keras
from keras.applications import VGG16

# Load VGG16 pre-trained on ImageNet data
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional layers
for layer in vgg_model.layers:
    layer.trainable = False

# Create a Sequential model and add the VGG16 base
model = Sequential()
model.add(vgg_model)

# Flatten the output of VGG16
model.add(Flatten())

# Add dense layers for classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Display the summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               4194816   
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                       

EfficientNet

In [ ]:
# Import EfficientNetB0 from Keras
from keras.applications import EfficientNetB0

# Load EfficientNetB0 pre-trained on ImageNet data
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers
for layer in efficientnet_model.layers:
    layer.trainable = False

# Create a Sequential model and add the EfficientNetB0 base
model = Sequential()
model.add(efficientnet_model)

# Flatten the output of EfficientNetB0
model.add(Flatten())

# Add dense layers for classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Display the summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")


16705208/16705208 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 5, 5, 1280)        4049571   
 )                                                               
                                                                 
 flatten_2 (Flatten)         (None, 32000)             0         
                                                                 
 dense_6 (Dense)             (None, 512)               16384512  
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               262656    
                                                                 
 dropout_9 (Dropout)         (None, 512)               

Xception

In [ ]:
# Import Xception from Keras
from keras.applications import Xception

# Load Xception pre-trained on ImageNet data
xception_model = Xception(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers
for layer in xception_model.layers:
    layer.trainable = False

# Create a Sequential model and add the Xception base
model = Sequential()
model.add(xception_model)

# Flatten the output of Xception
model.add(Flatten())

# Add dense layers for classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Display the summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")


83683744/83683744 [==============================] - 0s 0us/step
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 flatten_3 (Flatten)         (None, 51200)             0         
                                                                 
 dense_9 (Dense)             (None, 512)               26214912  
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                       

MobileNet

In [ ]:
# Import MobileNet from Keras
from keras.applications import MobileNet

# Load MobileNet pre-trained on ImageNet data
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the layers
for layer in mobilenet_model.layers:
    layer.trainable = False

# Create a Sequential model and add the MobileNet base
model = Sequential()
model.add(mobilenet_model)

# Flatten the output of MobileNet
model.add(Flatten())

# Add dense layers for classification
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Display the summary of the model
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")


Accuracy Plot

In [ ]:
#Accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(10,7))
plt.plot(epochs,acc,'r',label="Training Accuracy")
plt.plot(epochs,val_acc,'b',label="Test Accuracy")
plt.legend(loc='upper left')
plt.show()


Loss Plot

In [ ]:
#loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
fig = plt.figure(figsize=(10,7))
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Test loss")
plt.legend(loc='upper left')
plt.show()

Prediction Model

In [ ]:
# Assuming you have a mapping like this
folder_mapping = {
    0: 'glioma_tumor',
    1: 'meningioma_tumor',
    2: 'no_tumor',
    3: 'pituitary_tumor',
}

img = cv2.imread('/content/drive/MyDrive/input/Testing/pituitary_tumor/image(41).jpg')
img = cv2.resize(img, (150, 150))
img_array = np.array(img)
img_array = img_array.reshape(1, 150, 150, 3)

img = image.load_img('/content/drive/MyDrive/input/Testing/pituitary_tumor/image(41).jpg')
plt.imshow(img, interpolation='nearest')
plt.show()

a = model.predict(img_array)
indices = a.argmax()

# Print the folder name based on the index
folder_name = folder_mapping.get(indices, "Unknown Folder")
print("The Tumor is : ", folder_name)
